In [7]:
import os
from pytse_client.download import download_financial_indexes
import pandas as pd
import jalali_pandas
import math
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [8]:
def return_calculator(df):
    period_return = 1
    li = list(df)
    for day, _ in df.iteritems():
        idx = df.index.get_loc(day)
        elem = li[idx]
        if (math.isnan(elem)):
            elem = outlier_handler(elem)
            if elem == False:
                continue
        period_return = period_return * (1+elem)
    period_return = period_return - 1
    return period_return

def outlier_handler(ret):
    if (math.isnan(ret)):
        return False


In [9]:
desktop = os.path.join(os.path.join(os.path.expanduser('~')), 'Desktop') 
bourse_view = os.path.join(desktop, "sector_slc", "bourse_view")
sectors_data_path = os.path.join(bourse_view, "sectors")
market_data_path = os.path.join(bourse_view, "market")
list_of_sector_files = os.listdir(sectors_data_path)
list_of_market_files = os.listdir(market_data_path)
market_data_processed_path = os.path.join(bourse_view, "market_data_processed")
sectors_data_processed_path = os.path.join(bourse_view, "sectors_data_processed")
list_of_sector_processed_files = os.listdir(sectors_data_processed_path)
list_of_sector_processed_files = list_of_sector_processed_files[1:]
list_of_market_processed_files = os.listdir(market_data_processed_path)
list_of_market_processed_files = list_of_market_processed_files[1:]
dataset_path = os.path.join(bourse_view, "dataset")

In [13]:
for file_name in list_of_sector_files:
    
    data = pd.read_excel(sectors_data_path+ "\\" + file_name)
    first_row_with_pe_ttm = data[data["P/E-ttm"] != "-"].iloc[-1].name 
    data = data[:first_row_with_pe_ttm + 1]
    first_row_with_trade_value = data[data["ارزش معاملات"] != "-"].iloc[-1].name 
    data = data[:first_row_with_trade_value + 1]
    first_row_with_pct_change = data[data["درصد تغییر"] != "-"].iloc[-1].name 
    data = data[:first_row_with_pct_change + 1]
    data = data[["تاریخ شمسی","درصد تغییر","ارزش معاملات","P/E-ttm"]]
    data = data.replace('-', np.nan)
    data["date"] = data["تاریخ شمسی"].jalali.parse_jalali("%Y-%m-%d")
    data["month"] = data["date"].jalali.month
    data["year"] = data["date"].jalali.year
    data["quarter"] = data["date"].jalali.quarter
    data["weekday"] = data["date"].jalali.weekday
    data["day"] = data["date"].jalali.day
    data = data.drop(["تاریخ شمسی"],axis=1)
    data.rename(columns = {'درصد تغییر':'dayli_return', 'ارزش معاملات':'trade_value'}, inplace = True)
    last_years_month = data["month"][0]
    last_year = data["year"][0]
    first_years_month = data["month"].iloc[-1]
    first_year = data["year"].iloc[-1]

    empty_li_month = []
    empty_li_year = []
    for year in range(first_year, last_year + 1):

        if year == first_year:
            empty_li_month += [i for i in range(first_years_month,13)]
            empty_li_year += [year] * (13-first_years_month)

        elif year == last_year:
            empty_li_month += [i for i in range(1,last_years_month+1)]
            empty_li_year += [year] * (last_years_month)

        else:
            empty_li_month += [i for i in range(1,13)]
            empty_li_year += [year] * 12

    monthly_data = pd.DataFrame(columns=["year","quarter","month","monthly_return","monthly_trade_value","P/E-ttm"])
    monthly_data["year"] = empty_li_year
    monthly_data["month"] = empty_li_month

    for year in range(first_year, last_year + 1):

        if year == first_year:
            for month in range(first_years_month, 13):
                idx = monthly_data[(monthly_data.year == year) & (monthly_data.month == month)].index.item()
                ret = return_calculator(data[(data["month"] == month) & (data["year"] == year)]["dayli_return"])
                monthly_trade_value = data[(data["month"] == month) & (data["year"] == year)]["trade_value"].mean()
                month_pe_ttm = data[(data["month"] == month) & (data["year"] == year)]["P/E-ttm"].dropna().mean()
                monthly_data["quarter"][idx] = data[(data["month"] == month) & (data["year"] == year)].quarter.mean()
                monthly_data["monthly_return"][idx] = ret
                monthly_data["monthly_trade_value"][idx] = monthly_trade_value
                monthly_data["P/E-ttm"][idx] = month_pe_ttm

        elif year == last_year:
            for month in range(1, last_years_month + 1):
                idx = monthly_data[(monthly_data.year == year) & (monthly_data.month == month)].index.item()
                ret = return_calculator(data[(data["month"] == month) & (data["year"] == year)]["dayli_return"])
                monthly_trade_value = data[(data["month"] == month) & (data["year"] == year)]["trade_value"].mean()
                month_pe_ttm = data[(data["month"] == month) & (data["year"] == year)]["P/E-ttm"].dropna().mean()
                monthly_data["quarter"][idx] = data[(data["month"] == month) & (data["year"] == year)].quarter.mean()
                monthly_data["monthly_return"][idx] = ret
                monthly_data["monthly_trade_value"][idx] = monthly_trade_value
                monthly_data["P/E-ttm"][idx] = month_pe_ttm

        else:
            for month in range(1, 13):
                idx = monthly_data[(monthly_data.year == year) & (monthly_data.month == month)].index.item()
                ret = return_calculator(data[(data["month"] == month) & (data["year"] == year)]["dayli_return"])
                monthly_trade_value = data[(data["month"] == month) & (data["year"] == year)]["trade_value"].mean()
                month_pe_ttm = data[(data["month"] == month) & (data["year"] == year)]["P/E-ttm"].dropna().mean()
                monthly_data["quarter"][idx] = data[(data["month"] == month) & (data["year"] == year)].quarter.mean()
                monthly_data["monthly_return"][idx] = ret
                monthly_data["monthly_trade_value"][idx] = monthly_trade_value
                monthly_data["P/E-ttm"][idx] = month_pe_ttm

    monthly_data.to_excel(sectors_data_processed_path + "\\" + file_name)

market_df = pd.DataFrame()
for file_name in list_of_market_files:
    data = pd.read_excel(market_data_path+ "\\" + file_name)
    file_name = file_name.split(".")[0]
    data.rename(columns = {'مقدار':file_name, "تاریخ شمسی":"date"}, inplace = True)
    data = data.drop(["تاریخ میلادی"],axis=1)
    data["date"] = data["date"].jalali.parse_jalali("%Y-%m-%d")
    data["month"] = data["date"].jalali.month
    data["year"] = data["date"].jalali.year
    data["quarter"] = data["date"].jalali.quarter
    data["weekday"] = data["date"].jalali.weekday
    data["day"] = data["date"].jalali.day
    if len(market_df) == 0:
        market_df = data
    else:
        market_df = market_df.merge(data,on="date",how="inner")
    
market_df = market_df[["date","market_pe","overall_index","trade_value","usd"]]
market_df["month"] = market_df["date"].jalali.month
market_df["year"] = market_df["date"].jalali.year
market_df["quarter"] = market_df["date"].jalali.quarter
market_df["weekday"] = market_df["date"].jalali.weekday
market_df["day"] = market_df["date"].jalali.day
market_df["usd"] = market_df["usd"].pct_change() 
market_df["overall_index"] = market_df["overall_index"].pct_change()

first_years_month = market_df["month"][0]
first_year = market_df["year"][0]
last_years_month = market_df["month"].iloc[-1]
last_year = market_df["year"].iloc[-1]

empty_li_month = []
empty_li_year = []
for year in range(first_year, last_year + 1):

    if year == first_year:
        empty_li_month += [i for i in range(first_years_month,13)]
        empty_li_year += [year] * (13-first_years_month)

    elif year == last_year:
        empty_li_month += [i for i in range(1,last_years_month+1)]
        empty_li_year += [year] * (last_years_month)

    else:
        empty_li_month += [i for i in range(1,13)]
        empty_li_year += [year] * 12

monthly_market_data = pd.DataFrame(columns=["year","quarter","month","index_monthly_return","monthly_trade_value","monthly_usd_return","market_pe"])
monthly_market_data["year"] = empty_li_year
monthly_market_data["month"] = empty_li_month

for year in range(first_year, last_year + 1):

    if year == first_year:
        for month in range(first_years_month, 13):
            idx = monthly_market_data[(monthly_market_data.year == year) & (monthly_market_data.month == month)].index.item()
            index_ret = return_calculator(market_df[(market_df["month"] == month) & (market_df["year"] == year)]["overall_index"])
            usd_ret = return_calculator(market_df[(market_df["month"] == month) & (market_df["year"] == year)]["usd"])
            monthly_trade_value = market_df[(market_df["month"] == month) & (market_df["year"] == year)]["trade_value"].mean()
            month_pe_ttm = market_df[(market_df["month"] == month) & (market_df["year"] == year)]["market_pe"].dropna().mean()
            monthly_market_data["quarter"][idx] = market_df[(market_df["month"] == month) & (market_df["year"] == year)].quarter.mean()
            monthly_market_data["index_monthly_return"][idx] = index_ret
            monthly_market_data["monthly_usd_return"][idx] = usd_ret
            monthly_market_data["monthly_trade_value"][idx] = monthly_trade_value
            monthly_market_data["market_pe"][idx] = month_pe_ttm

    elif year == last_year:
        for month in range(1, last_years_month + 1):
            idx = monthly_market_data[(monthly_market_data.year == year) & (monthly_market_data.month == month)].index.item()
            index_ret = return_calculator(market_df[(market_df["month"] == month) & (market_df["year"] == year)]["overall_index"])
            usd_ret = return_calculator(market_df[(market_df["month"] == month) & (market_df["year"] == year)]["usd"])
            monthly_trade_value = market_df[(market_df["month"] == month) & (market_df["year"] == year)]["trade_value"].mean()
            month_pe_ttm = market_df[(market_df["month"] == month) & (market_df["year"] == year)]["market_pe"].dropna().mean()
            monthly_market_data["quarter"][idx] = market_df[(market_df["month"] == month) & (market_df["year"] == year)].quarter.mean()
            monthly_market_data["index_monthly_return"][idx] = index_ret
            monthly_market_data["monthly_usd_return"][idx] = usd_ret
            monthly_market_data["monthly_trade_value"][idx] = monthly_trade_value
            monthly_market_data["market_pe"][idx] = month_pe_ttm

    else:
        for month in range(1, 13):
            idx = monthly_market_data[(monthly_market_data.year == year) & (monthly_market_data.month == month)].index.item()
            index_ret = return_calculator(market_df[(market_df["month"] == month) & (market_df["year"] == year)]["overall_index"])
            usd_ret = return_calculator(market_df[(market_df["month"] == month) & (market_df["year"] == year)]["usd"])
            monthly_trade_value = market_df[(market_df["month"] == month) & (market_df["year"] == year)]["trade_value"].mean()
            month_pe_ttm = market_df[(market_df["month"] == month) & (market_df["year"] == year)]["market_pe"].dropna().mean()
            monthly_market_data["quarter"][idx] = market_df[(market_df["month"] == month) & (market_df["year"] == year)].quarter.mean()
            monthly_market_data["index_monthly_return"][idx] = index_ret
            monthly_market_data["monthly_usd_return"][idx] = usd_ret
            monthly_market_data["monthly_trade_value"][idx] = monthly_trade_value
            monthly_market_data["market_pe"][idx] = month_pe_ttm

monthly_market_data.to_csv(market_data_processed_path + "\\" + "market_data_processed.csv")



dataset_df = pd.DataFrame()
for file_name in list_of_sector_processed_files:
    data = pd.read_excel(sectors_data_processed_path+ "\\" + file_name)
    data = data.fillna(method='ffill')

    first_year_quarter = data.quarter.iloc[0]
    first_year = data.year.iloc[0]
    last_year_quarter = data.quarter.iloc[-1]
    last_year = data.year.iloc[-1]

    empty_li_quarters = []
    empty_li_year = []

    for year in range(first_year, last_year + 1):

        if year == first_year:
            empty_li_quarters += [i for i in range(first_year_quarter,5)]
            empty_li_year += [year] * (5-first_year_quarter)

        elif year == last_year:
            empty_li_quarters += [i for i in range(1,last_year_quarter+1)]
            empty_li_year += [year] * (last_year_quarter)

        else:
            empty_li_quarters += [i for i in range(1,5)]
            empty_li_year += [year] * 4

    quarterly_data = pd.DataFrame(columns=["year","quarter","quarterly_return","quarterly_trade_value","P/E-ttm"])
    quarterly_data["year"] = empty_li_year
    quarterly_data["quarter"] = empty_li_quarters

    for year in range(first_year, last_year + 1):

        if year == first_year:
            for quarter in range(first_year_quarter, 5):
                idx = quarterly_data[(quarterly_data.year == year) & (quarterly_data.quarter == quarter)].index.item()
                ret = return_calculator(data[(data["quarter"] == quarter) & (data["year"] == year)]["monthly_return"])
                pe = data[(data["quarter"] == quarter) & (data["year"] == year)]["P/E-ttm"].iloc[-1]
                trade_value = data[(data["quarter"] == quarter) & (data["year"] == year)]["monthly_trade_value"].mean()
                quarterly_data["quarterly_return"][idx] = ret
                quarterly_data["quarterly_trade_value"][idx] = trade_value
                quarterly_data["P/E-ttm"][idx] = pe

        elif year == last_year:
            for quarter in range(1, last_year_quarter + 1):
                idx = quarterly_data[(quarterly_data.year == year) & (quarterly_data.quarter == quarter)].index.item()
                ret = return_calculator(data[(data["quarter"] == quarter) & (data["year"] == year)]["monthly_return"])
                pe = data[(data["quarter"] == quarter) & (data["year"] == year)]["P/E-ttm"].iloc[-1]
                trade_value = data[(data["quarter"] == quarter) & (data["year"] == year)]["monthly_trade_value"].mean()
                quarterly_data["quarterly_return"][idx] = ret
                quarterly_data["quarterly_trade_value"][idx] = trade_value
                quarterly_data["P/E-ttm"][idx] = pe
        else:
            for quarter in range(1, 5):
                idx = quarterly_data[(quarterly_data.year == year) & (quarterly_data.quarter == quarter)].index.item()
                ret = return_calculator(data[(data["quarter"] == quarter) & (data["year"] == year)]["monthly_return"])
                pe = data[(data["quarter"] == quarter) & (data["year"] == year)]["P/E-ttm"].iloc[-1]
                trade_value = data[(data["quarter"] == quarter) & (data["year"] == year)]["monthly_trade_value"].mean()
                quarterly_data["quarterly_return"][idx] = ret
                quarterly_data["quarterly_trade_value"][idx] = trade_value
                quarterly_data["P/E-ttm"][idx] = pe

    quarterly_data["P/E-ttm_1_lag"] = quarterly_data["P/E-ttm"].shift(1)
    quarterly_data["P/E-ttm_2_lag"] = quarterly_data["P/E-ttm"].shift(2)
    quarterly_data["quarterly_return_1_lag"] = quarterly_data["quarterly_return"].shift(1)
    quarterly_data["quarterly_return_2_lag"] = quarterly_data["quarterly_return"].shift(2)
    quarterly_data = quarterly_data.dropna().reset_index().drop(["index"],axis=1)

    data = pd.read_csv(market_data_processed_path+ "\\" + list_of_market_processed_files[0])
    data = data.drop(["Unnamed: 0"],axis=1)
    data = data.fillna(method='ffill')
    empty_li_quarters = []
    empty_li_year = []

    for year in range(first_year, last_year + 1):

        if year == first_year:
            empty_li_quarters += [i for i in range(first_year_quarter,5)]
            empty_li_year += [year] * (5-first_year_quarter)

        elif year == last_year:
            empty_li_quarters += [i for i in range(1,last_year_quarter+1)]
            empty_li_year += [year] * (last_year_quarter)

        else:
            empty_li_quarters += [i for i in range(1,5)]
            empty_li_year += [year] * 4

    market_quarterly_data = pd.DataFrame(columns=["year","quarter","index_quarterly_return","usd_quarterly_return","quarterly_trade_value","market_pe"])
    market_quarterly_data["year"] = empty_li_year
    market_quarterly_data["quarter"] = empty_li_quarters

    for year in range(first_year, last_year + 1):

        if year == first_year:
            for quarter in range(first_year_quarter, 5):
                idx = market_quarterly_data[(market_quarterly_data.year == year) & (market_quarterly_data.quarter == quarter)].index.item()
                index_ret = return_calculator(data[(data["quarter"] == quarter) & (data["year"] == year)]["index_monthly_return"])
                usd_ret = return_calculator(data[(data["quarter"] == quarter) & (data["year"] == year)]["monthly_usd_return"])
                pe = data[(data["quarter"] == quarter) & (data["year"] == year)]["market_pe"].iloc[-1]
                trade_value = data[(data["quarter"] == quarter) & (data["year"] == year)]["monthly_trade_value"].mean()
                market_quarterly_data["index_quarterly_return"][idx] = index_ret
                market_quarterly_data["usd_quarterly_return"][idx] = usd_ret
                market_quarterly_data["quarterly_trade_value"][idx] = trade_value
                market_quarterly_data["market_pe"][idx] = pe

        elif year == last_year:
            for quarter in range(1, last_year_quarter + 1):
                idx = market_quarterly_data[(market_quarterly_data.year == year) & (market_quarterly_data.quarter == quarter)].index.item()
                index_ret = return_calculator(data[(data["quarter"] == quarter) & (data["year"] == year)]["index_monthly_return"])
                usd_ret = return_calculator(data[(data["quarter"] == quarter) & (data["year"] == year)]["monthly_usd_return"])
                pe = data[(data["quarter"] == quarter) & (data["year"] == year)]["market_pe"].iloc[-1]
                trade_value = data[(data["quarter"] == quarter) & (data["year"] == year)]["monthly_trade_value"].mean()
                market_quarterly_data["index_quarterly_return"][idx] = index_ret
                market_quarterly_data["usd_quarterly_return"][idx] = usd_ret
                market_quarterly_data["quarterly_trade_value"][idx] = trade_value
                market_quarterly_data["market_pe"][idx] = pe
        else:
            for quarter in range(1, 5):
                idx = market_quarterly_data[(market_quarterly_data.year == year) & (market_quarterly_data.quarter == quarter)].index.item()
                index_ret = return_calculator(data[(data["quarter"] == quarter) & (data["year"] == year)]["index_monthly_return"])
                usd_ret = return_calculator(data[(data["quarter"] == quarter) & (data["year"] == year)]["monthly_usd_return"])
                pe = data[(data["quarter"] == quarter) & (data["year"] == year)]["market_pe"].iloc[-1]
                trade_value = data[(data["quarter"] == quarter) & (data["year"] == year)]["monthly_trade_value"].mean()
                market_quarterly_data["index_quarterly_return"][idx] = index_ret
                market_quarterly_data["usd_quarterly_return"][idx] = usd_ret
                market_quarterly_data["quarterly_trade_value"][idx] = trade_value
                market_quarterly_data["market_pe"][idx] = pe

    market_quarterly_data["index_quarterly_return_1_lag"] = market_quarterly_data["index_quarterly_return"].shift(1)
    market_quarterly_data["index_quarterly_return_2_lag"] = market_quarterly_data["index_quarterly_return"].shift(2)
    market_quarterly_data["usd_quarterly_return_1_lag"] = market_quarterly_data["usd_quarterly_return"].shift(1)
    market_quarterly_data["usd_quarterly_return_2_lag"] = market_quarterly_data["usd_quarterly_return"].shift(2)
    market_quarterly_data["market_pe_1_lag"] = market_quarterly_data["market_pe"].shift(1)
    market_quarterly_data["market_pe_2_lag"] = market_quarterly_data["market_pe"].shift(2)
    market_quarterly_data = market_quarterly_data.dropna().reset_index().drop(["index"],axis=1)

    quarterly_data["quarterly_usd_return"] = market_quarterly_data["usd_quarterly_return"]
    quarterly_data["quarterly_usd_return_1_lag"] = market_quarterly_data["usd_quarterly_return_1_lag"]
    quarterly_data["quarterly_usd_return_2_lag"] = market_quarterly_data["usd_quarterly_return_2_lag"]
    quarterly_data["market_pe"] = market_quarterly_data["market_pe"]
    quarterly_data["market_pe_1_lag"] = market_quarterly_data["market_pe_1_lag"]
    quarterly_data["market_pe_2_lag"] = market_quarterly_data["market_pe_2_lag"]
    quarterly_data["relative_trade_value"] = 100 * quarterly_data["quarterly_trade_value"] / market_quarterly_data["quarterly_trade_value"]
    quarterly_data["index_quarterly_return"] = market_quarterly_data["index_quarterly_return"]
    quarterly_data["index_quarterly_return_1_lag"] = market_quarterly_data["index_quarterly_return_1_lag"]
    quarterly_data["index_quarterly_return_2_lag"] = market_quarterly_data["index_quarterly_return_2_lag"]
    quarterly_data["quarter_excess_return"] = quarterly_data["quarterly_return"] - quarterly_data["index_quarterly_return"]
    quarterly_data["sector"] = [file_name.split(".")[0]] * len(quarterly_data) 

    if len(dataset_df) == 0:
        dataset_df = quarterly_data

    else:
        dataset_df = pd.concat([dataset_df,quarterly_data])

dataset_df = dataset_df.sort_values(by = ['year', 'quarter'], ascending = [True, True])
dataset_df.to_csv(dataset_path + "\\" + "dataset.csv")